## 1초에 100프레임으로 타겟을 추적할 수 있는 GoTurn
### Learning to Track at 100 FPS with Deep Regression Networks 

이번에 리뷰할 논문은 2016년(ECCV)에 나온 논문으로, 코드가 C++ 로 공개되어 있습니다. 

프로젝트 페이지는: http://davheld.github.io/GOTURN/GOTURN.html 

논문 링크는: http://davheld.github.io/GOTURN/GOTURN.pdf 입니다.

모델의 구조 자체가 굉장히 간단해서 Pytorch 로도 쉽게 구현 할 수 있습니다.

<img src=\"images/model.png\" width=\"800\" height=\"500\" />\n

** - 제시하는 문제? ** 


머신러닝을 사용하는 Object Tracking 모델들은 주로 Online 에서 트레이닝 하는 방법을 사용해서 방대한 데이터를 사용할 수 없습니다. 
이 논문에서는 오프라인에서 뉴럴네트워크를 학습시켜서 1초에 100프레임 (100FPS)으로 작동할 수 있는 모델을 제시했습니다.

** - 어떻게? **

간단한 Feed-Forward Network를 이용한 모델입니다.물체의 움직임과 appearance 사이의 관계를 학습해서 training set에 나타나지 않은 새로운 물체를 추적할 수 있게 했습니다. 

### 1. Intro

** - Single Target Tracking 이란? ** 


비디오의 한 프레임이 제시되었을 때, 관심있는 Target 에 대해서 후속 비디오 프레임에서 이 target을 계속 찾는 것 입니다. 

single tracking이 기반이 되어서 자율주행의 obstacle tracking 등 다양한 시스템을 구성할 수 있습니다. 

** - GOTURN? ** 

Generic Object Tracking Using Regression Networks 의 약자로, 오프라인에서 모델을 비디오로 학습시켜서 generic 한 물체를 트래킹 할 수 있도록 만들어졌습니다. 

위 모델의 가장 강점은 offline에서 training 시킬 수 있어서 100PFS로 빠르게 사용할 수 있다는 것입니다. 

** - 그렇다면, 왜 이전 모델들은 느렸는가? ** 


  - Online 에서 트레이닝을 시켰다
  - 주로 Classification-based 접근을 사용: 많은 image patch 를 만들어서 target을 찾는 방법으로 접근 
    --> 여기서는 Regression-based 접근 사용: single feed-forward pass 를 이용해서 타겟의 위치로 바로 찾을 수 있도록 한다 

### 2. Method

#### Input & Output
- Target 물체가 무엇인지 이미지 제시: 타겟이 이미지의 중심에 오도록 전 프레임에 있는 타겟 이미지를 crop 한다 
    - 타겟에 대한 정보를 전 프레임에서 가져오기 때문에 처음 본 타겟도 잘 찾을 수 있다 
    - Crop 된 이미지 안에 있는 타겟을 찾도록 네트워크가 설정된다 
    - 타겟 주변의 contextual information도 제공하기 위해서 어느정도 패드를 넣어준다 


프레임 (t - 1) 일 때 네트워크가 예상한 bounding box의 위치를 c = (cx, cy) 라고 하자. 
프레임 t가 됐을 때, 전 프레임에서 예상한 위치 c 를 중심으로 crop 을 또 하면서 다음 프레임에게 타겟이 무엇인지 알려준다. 

** - 타겟의 위치를 어떻게 예상하는가? **

전 프레임에서 타겟이 어디에 있었는지 본다. 
보통 물체들이 Smooth 하게 움직이기 때문에 다음 프레임에서 물체가 어디쯤에 있을지 좋은 힌트가 된다. 

전 프레임에서 예측한 그 위치에 전보다 k배 큰 박스를 만들어서 Crop 한다 (k = 2를 사용함. 즉, 2배 큰 박스를 만든다). 

타겟이 너무 빨리 움직이거나 다른 물체에 의해서 감춰지지 않는 이상 crop  된 박스 안에 물체가 있으 것이다. 빨리 움직이는 물체라면, k 를 증가시킨다. 

** - Output: 현재 프레임에서 물체가 있을 것이라고 예상되는 곳의 coordinates ** 


### Network Architecture 

인풋: 타겟, Search region 

Convolutional Layer --> Fully Connected Layers 


- Convolutional Layer 의 역할: 이미지의 high-level representation 뽑아내기 
- Fully Connected Layer 의 역할: 타겟의 Feature와 현재 프레임의 feature 를 비교해서 어디있는지 예측하는 것 


1.  Convolutional Layer:  CaffeNet의 첫 5개 Layer 

2.  여기서 나온 결과를 하나의 벡터로 연결한다 

3.  이 벡터를 3개의 Fully Connected Layer에 넣는다 (4096nodes) 

4.  Output Layer: 4 nodes 인 Fully Connected Layer 


CaffeNet의 parameter를 사용하고, fully connected layer 사이에 dropout, ReLU 사용. 


---


### 정리하면, 
- 처음 인풋으로 어떤걸 타겟으로 할 것인지 직접 타겟 주변에 bounding box를 만들어서 입력해야 한다 

- 내가 입력한 bounding box만 crop 해서 #1 convolution -> 그 타겟의 feature 를 뽑아내기 
 
 
- 그 bounding box 보다 k 배 큰 박스를 다음 프레임에 같은 위치에서 crop! (보통 smooth하게 움직여서 주변에 있을 확률이 높음) 빨리 움직이는 물체가 타겟이라면 k를 크게 하면 됨 --> 이 crop 된 이미지도 #2 convolution -> 그 이미지의 feature 뽑아내기

- 두 Feature map을 비교해서 타겟의 feature 와 같은 곳에 새로운 bounding box 를 만든다 

- 무한반복 

### Learning Motion Smoothness 

보통 물체는 smooth 하게 움직이기 때문에 

큰 움직임 보다는 작은 움직임을 prefer하도록 모델을 가르쳐야 한다. 

HOW? 
- Laplace distribution으 사용해서 random crop을 한 training set를 augment 시킨다. 

### 그 외에 ... 
- convolution layer: pretrained on ImageNet 
- No fine-tuning - 트레이닝 사이즈가 작기때문에 못함 
- Nvidia GeForce GTX Titan X GPU (cuDNN acceleration): 165fps 
- GTX 680 GPU: 100fps
- CPU: 2.7 fps

결과: [사진 넣기]



In [2]:
import torch
from torchvision import models 
import torch.nn as nn

# Most of the source code: https://github.com/amoudgl/pygoturn/blob/master/src/model.py 

class GoNet(nn.Module):
    def __init__(self):
        
        #nn.Module 에서 상속받도록 
        super(GoNet, self).__init__()
        
        #Convolution 으로 학습된 caffenet을 그대로 썼으니까 가져오기 
        caffenet = models.alexnet(pretrained=True)
        for param in caffenet.parameters():
            param.requires_grad = False  #Fine tuning 이 안되도록 모든 layer freeze 시키기
        self.features = caffenet.features
        
        #Caffenet과 같은 구조로 만들기 
        self.classifier = nn.Sequential(
                nn.Linear(256*6*6*2, 4096),
                nn.ReLU(inplace=True),
                nn.Dropout(),
                nn.Linear(4096, 4096),
                nn.ReLU(inplace=True),
                nn.Dropout(),
                nn.Linear(4096,4096),
                nn.ReLU(inplace=True),
                nn.Dropout(),
                nn.Linear(4096, 4),
                )
        
        self.weight_init()
    
    def weight_init(self):
        for m in self.classifier.modules():
            # Fully Connected Layer의 parameter 들은 mean=0, std=0.005, bias=1로 initialize 됨
            if isinstance(m, nn.Linear):
                m.bias.data.fill_(1)
                m.weight.data.normal_(0, 0.005)

    #실제로 Model 만들기!
    def forward(self, x, y):
        #각기 다른 2개의 stream 을 만들고
        #합쳐(concat해)서 fully connected 에 넣기
        x1 = self.features(x)
        x1 = x1.view(x.size(0), 256*6*6)
        
        x2 = self.features(y)
        x2 = x2.view(x.size(0), 256*6*6)
        
        x = torch.cat((x1, x2), 1)
        x = self.classifier(x)
        return x

#### 출처: https://github.com/amoudgl/pygoturn/blob/master

사실 이 GoTurn 모델 자체는 굉장히 간단하지만, 데이터 전처리 작업이 훨씬 오래걸리는 모델입니다. 

Convolution 에 데이터를 넣기 전에 initial Bounding Box 를 찾는 것, 

다음 프레임에 사용될 predicted bounding box 를 예측하는 것 등 convolution에 들어가기 전까지가 더 머리아픈 것 같습니다 ㅠㅡㅜ 

코드의 출처를 보시면 구현을 해놓았으니, 관심있으면 확인하세요 :) 